<a href="https://colab.research.google.com/github/fralfaro/MAT281_2022/blob/main/labs/lab_07/lab_07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAT281 - Laboratorio N°07


<a id='p1'></a>
## I.- Problema 01


<img src="https://www.cardrates.com/wp-content/uploads/2020/08/shutterstock_576998230.jpg?1" width="480" height="360" align="center"/>


El conjunto de datos se denomina `creditcard.csv` y consta de varias columnas con información acerca del fraude de tarjetas de crédito, en donde la columna **Class** corresponde a: 0 si no es un fraude y 1 si es un fraude.

En este ejercicio se trabajará el problemas de  clases desbalancedas. Veamos las primeras cinco filas dle conjunto de datos:

In [29]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn.model_selection import train_test_split

from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

%matplotlib inline
sns.set_palette("deep", desat=.6)
sns.set(rc={'figure.figsize':(11.7,8.27)})

In [30]:
# cargar datos
df = pd.read_csv("https://raw.githubusercontent.com/fralfaro/MAT281_2022/main/labs/lab_07/data/creditcard.csv",sep=';')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
2,10.0,1.249999,-1.221637,0.383930,-1.234899,-1.485419,-0.753230,-0.689405,-0.227487,-2.094011,...,-0.231809,-0.483285,0.084668,0.392831,0.161135,-0.354990,0.026416,0.042422,121.50,0
3,10.0,0.384978,0.616109,-0.874300,-0.094019,2.924584,3.317027,0.470455,0.538247,-0.558895,...,0.049924,0.238422,0.009130,0.996710,-0.767315,-0.492208,0.042472,-0.054337,9.99,0
4,12.0,1.103215,-0.040296,1.267332,1.289091,-0.735997,0.288069,-0.586057,0.189380,0.782333,...,-0.024612,0.196002,0.013802,0.103758,0.364298,-0.382261,0.092809,0.037051,12.99,0


Analicemos el total de fraudes respecto a los casos que nos son fraudes:


In [31]:
# calcular proporciones
df_count = pd.DataFrame()
df_count["fraude"] =["no","si"]
df_count["total"] = df["Class"].value_counts() 
df_count["porcentaje"] = 100*df_count["total"] /df_count["total"] .sum()

df_count

,fraude,total,porcentaje
0,no,50000,99.025588
1,si,492,0.974412


Se observa que menos del 1% corresponde a registros frudulentos. La pregunta que surgen son:

* ¿ Cómo deben ser el conjunto de entrenamiento y de testeo?
* ¿ Qué modelos ocupar?
* ¿ Qué métricas ocupar?

Por ejemplo, analicemos el modelos de regresión logística y apliquemos el procedimiento estándar:

In [32]:
# datos 
y = df.Class
X = df.drop('Class', axis=1)

# split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=27)


# Creando el modelo
lr = LogisticRegression(solver='liblinear').fit(X_train, y_train)
 
# predecir
lr_pred = lr.predict(X_test)

# calcular accuracy
accuracy_score(y_test, lr_pred)

0.9961181969420898

En general el modelo tiene un **accuracy** del 99,9%, es decir, un podría suponer que el modelo predice casi perfectamente, pero eso esta lejos de ser así.  Para ver por qué es necesario seguir los siguientes pasos:

### 1. Cambiar la métrica de rendimiento

El primer paso es comparar con distintas métricas, para eso ocupemos las 4 métricas clásicas abordadas en el curso:
* accuracy
* precision
* recall
* f-score

En este punto deberá poner las métricas correspondientes y comentar sus resultados.

In [33]:
# metrics
y_true =  list(y_test)
y_pred = list(lr.predict(X_test))

print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_test, lr_pred))
print('recall:     ',recall_score(y_test, lr_pred))
print('precision:  ',precision_score(y_test, lr_pred))
print('f-score:    ',f1_score(y_test, lr_pred))
print("")


Matriz de confusion:
 
[[12471    16]
 [   33   103]]

Metricas:
 
accuracy:    0.9961181969420898
recall:      0.7573529411764706
precision:   0.865546218487395
f-score:     0.807843137254902



Podemos observar que el **acurracy** es bastante alto pero esto es debido a que predice muy bien los verdaderos positivos pero no asi con los verdaderos negativos como se ve en la matriz de confusion, ahora para las demas metricas se observa que se identifica aprox. un **75%** de los fraudes , ademas se  tiene una **precisión** de aproximadamente el 86%  y  un 80% de **f.score** , lo cuales son valores relativamente elevados pero que aun pueden mejorar utilizando otros metodos para abordar el problema.

### 2. Cambiar algoritmo

El segundo paso es comparar con distintos modelos. Debe tener en cuenta que el modelo ocupaod resuelva el problema supervisado de clasificación.

En este punto deberá ajustar un modelo de **random forest**, aplicar las métricas y comparar con el modelo de regresión logística.

In [34]:
# train model

rfc =  RandomForestClassifier().fit(X_train, y_train)  # algoritmo random forest

In [35]:
# metrics

y_true =  list(y_test)
y_pred = list(rfc.predict(X_test)) # predicciones con random forest


print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_true,y_pred))
print('recall:     ',recall_score(y_true,y_pred))
print('precision:  ',precision_score(y_true,y_pred))
print('f-score:    ',f1_score(y_true,y_pred))
print("")


Matriz de confusion:
 
[[12484     3]
 [   25   111]]

Metricas:
 
accuracy:    0.9977818268240514
recall:      0.8161764705882353
precision:   0.9736842105263158
f-score:     0.888



A traves de la matriz de confusion podemos observar que los verdaderos negativos aumentaron, los verdaderos positivos y los falsos positivos disminuyeron por lo que este modelo seria mejor, tambien esto se ve reflejado en que todas las metricas aumentaron.

### 3. Técnicas de remuestreo: sobremuestreo de clase minoritaria

El tercer paso es ocupar ténicas de remuestreo, pero sobre la clase minoritaria. Esto significa que mediantes ténicas de remuestreo trataremos de equiparar el número de elementos de la clase minoritaria a la clase mayoritaria.

In [36]:
from sklearn.utils import resample

# concatenar el conjunto de entrenamiento
X = pd.concat([X_train, y_train], axis=1)

# separar las clases
not_fraud = X[X.Class==0]
fraud = X[X.Class==1]

# remuestrear  clase minoritaria
fraud_upsampled = resample(fraud,
                          replace=True, # sample with replacement
                          n_samples=len(not_fraud), # match number in majority class
                          random_state=27) # reproducible results

# recombinar resultados
upsampled = pd.concat([not_fraud, fraud_upsampled])

# chequear el número de elementos por clases
upsampled.Class.value_counts()

0    37513
1    37513
Name: Class, dtype: int64

In [37]:
# datos de entrenamiento sobre-balanceados
y_train = upsampled.Class
X_train = upsampled.drop('Class', axis=1)

Ocupando estos nuevos conjunto de entrenamientos, vuelva a aplicar el modelos de regresión logística y calcule las correspondientes métricas. Además, justifique las ventajas y desventjas de este procedimiento.

In [38]:
upsampled = LogisticRegression(solver='liblinear').fit(X_train,y_train) # algoritmo de regresion logistica

# metrics

y_true =  list(y_test)
y_pred = list(upsampled.predict(X_test))


print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_true,y_pred))
print('recall:     ',recall_score(y_true,y_pred))
print('precision:  ',precision_score(y_true,y_pred))
print('f-score:    ',f1_score(y_true,y_pred))
print("")


Matriz de confusion:
 
[[12200   287]
 [   12   124]]

Metricas:
 
accuracy:    0.976313079299691
recall:      0.9117647058823529
precision:   0.30170316301703165
f-score:     0.45338208409506403



Podemos observar que disminuyen los verdaderos positivos y aumentan los los verdaderos negativos pero se ve un aumento considerable en los falsos positivos, ademas de que la precision cayo a un 30% aproximadamente y como la presicion disminuyo tambien lo hizo el f-score, el accuracy sigue siendo elevado y se obtuvo un aumento en el recall , es decir, un aumento de la identificacion de los fraudes.

### 4. Técnicas de remuestreo - Ejemplo de clase mayoritaria

El cuarto paso es ocupar ténicas de remuestreo, pero sobre la clase mayoritaria. Esto significa que mediantes ténicas de remuestreo trataremos de equiparar el número de elementos de la clase mayoritaria  a la clase minoritaria.

In [39]:
# remuestreo clase mayoritaria
not_fraud_downsampled = resample(not_fraud,
                                replace = False, # sample without replacement
                                n_samples = len(fraud), # match minority n
                                random_state = 27) # reproducible results

# recombinar resultados
downsampled = pd.concat([not_fraud_downsampled, fraud])

# chequear el número de elementos por clases
downsampled.Class.value_counts()

0    356
1    356
Name: Class, dtype: int64

In [40]:
# datos de entrenamiento sub-balanceados

y_train = downsampled.Class
X_train = downsampled.drop('Class', axis=1)

Ocupando estos nuevos conjunto de entrenamientos, vuelva a aplicar el modelos de regresión logística y calcule las correspondientes métricas. Además, justifique las ventajas y desventjas de este procedimiento.

In [41]:
undersampled = LogisticRegression(solver='liblinear').fit(X_train,y_train) # modelo de regresi+on logística

# metrics

y_true =  list(y_test)
y_pred = list(undersampled.predict(X_test))


print('\nMatriz de confusion:\n ')
print(confusion_matrix(y_true,y_pred))

print('\nMetricas:\n ')
print('accuracy:   ',accuracy_score(y_true,y_pred))
print('recall:     ',recall_score(y_true,y_pred))
print('precision:  ',precision_score(y_true,y_pred))
print('f-score:    ',f1_score(y_true,y_pred))
print("")


Matriz de confusion:
 
[[12216   271]
 [   16   120]]

Metricas:
 
accuracy:    0.9772637249465261
recall:      0.8823529411764706
precision:   0.3069053708439898
f-score:     0.45540796963946867



Al igual que en la parte 3. disminuyen los verdederos positivos y aumentan los verdaderos negativos y tambien se ve un aumento en la cantidad de falsos positivos y con una precision de 30% aproximadamente, ademas de que el recall aumento pero en menor medida.

### 5. Conclusiones

Para finalizar el laboratorio, debe realizar un análisis comparativo con los disintos resultados obtenidos  en los pasos 1-4. Saque sus propias conclusiones del caso.

Se concluye que si se busca aumentar la cantidad de aciertos verdaderos negativos es recomendable utilizar tecnicas de muestreo como en la parte 3 y 4 teniendo en consideracion que eso aumentara el numero de falsos positivos , si se busca tener una mayor cantidad de aciertos verdaderos positvos lo mejor seria utilizar la parte 1 y por ultimo si busca un mejor resultado en terminos de métricas lo mejor seria utilizar Random forest.